In [1]:
import ai2thor.controller
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [2]:
# Choose a scene below
# Kitchens: FloorPlan1 - FloorPlan30
# Living rooms: FloorPlan201 - FloorPlan230
# Bedrooms: FloorPlan301 - FloorPlan330
# Bathrooms: FloorPLan401 - FloorPlan430
scene = 'FloorPlan204'

In [3]:
controller = ai2thor.controller.BFSController(grid_size=0.5)
controller.start(player_screen_width=400)
controller.search_all_closed(scene)
location = []
index = {}
i = 0
num_of_location = 0
for point in controller.grid_points:
    for k in range(4):
        location.append((point['x'], point['z']))
    index[(point['x'], point['z'])] = i
    i += 1
    num_of_location += 1
height = controller.grid_points[0]['y']

rotation = [0, 90, 180, 270] * len(controller.grid_points)
#  ------ location & index & rotation built ------
print('location & index & rotation built')

location & index & rotation built


In [4]:
def axis2list(string):
    for i in range(len(string)):
        if string[i] == '|':
            end = i
            break
    x = float(string[:end]) 
    y = float(string[end + 1:])
    return (x, y)

graph = []
for i in range(len(location) // 4):
    j = i * 4
    graph.append([-1,j+1,j+3,-1])
    graph.append([-1,j+2,j,-1])
    graph.append([-1,j+3,j+1,-1])
    graph.append([-1,j,j+2,-1])

G = controller.build_graph()
for g in G.nodes:
    x0, y0 = axis2list(g)
    for n in list(G[g]):
        x, y = axis2list(n)
        if x > x0:
            graph[index[x0,y0] * 4 + 1][0] = index[x,y] * 4 + 1
            graph[index[x0,y0] * 4 + 3][3] = index[x,y] * 4 + 3
        elif x < x0:
            graph[index[x0,y0] * 4 + 3][0] = index[x,y] * 4 + 3
            graph[index[x0,y0] * 4 + 1][3] = index[x,y] * 4 + 1            
        elif y > y0:
            graph[index[x0,y0] * 4 + 0][0] = index[x,y] * 4 + 0
            graph[index[x0,y0] * 4 + 2][3] = index[x,y] * 4 + 2             
        elif y < y0:
            graph[index[x0,y0] * 4 + 2][0] = index[x,y] * 4 + 2
            graph[index[x0,y0] * 4 + 0][3] = index[x,y] * 4 + 0
            
#  ------ graph built ------
print('graph built')

graph built


In [5]:
#### Graph: [Ahead, Right, Left, Back]
#print(graph[22])
#### Location: (x, y)
#print(location[22])
#### Rotation: [0, 90, 180, 270]
#print(len(rotation))
#### Edge in G
#print(G['-1.000|-1.000'])
#### index id with axis (need to multiply 4)
#print(index[-0.5,-1.5])

In [6]:
observation = []
# ----- look ahead -----
event = controller.step(dict(action='Teleport', x=location[0][0], y=height, z=location[0][1])) 
while event.metadata['agent']['rotation']['y'] != 0:
        event = controller.step(dict(action='RotateLeft'))       

for i in range(len(location) // 4):
    event = controller.step(dict(action='Teleport', x=location[i*4][0], y=height, z=location[i*4][1]))  
    observation.append(event.frame)
    event = controller.step(dict(action='RotateRight'))
    observation.append(event.frame)
    event = controller.step(dict(action='RotateRight'))
    observation.append(event.frame)
    event = controller.step(dict(action='RotateRight'))
    observation.append(event.frame)
    event = controller.step(dict(action='RotateRight'))

In [7]:
#print(len(observation))
#print(len(location))
#print(len(rotation))
#print(len(graph))
print('num of location:', len(index))
print('num of id', len(observation))

num of location: 34
num of id 136


In [8]:
shortest_path_distance = []
for i in range(len(location)):
    shortest_path_distance.append([])

def axis2string(axis):
    x = axis[0]
    y = axis[1]
    string = "%0.3f|%0.3f" % (x, y)
    return string

for i in range(len(location)):
    for j in range(len(location)):
        if (i,j) not in shortest_path_distance:
            turn = abs(i % 4 - j % 4)
            if turn == 3:
                turn = 1
            shortest_path_distance[i].append(turn + len(nx.shortest_path(G, axis2string(location[i]), axis2string(location[j]))) - 1)
#  ------ shortest_path_distance ------
print('shortest_path_distance built')
# Did not fully consider rotation, for access test

shortest_path_distance built


In [9]:
print(len(shortest_path_distance))

136


In [10]:
from tensorflow.contrib.slim.python.slim.nets import resnet_v1
import tensorflow as tf
import tensorflow.contrib.slim as slim

resnet_feature = []
inputs = tf.placeholder(tf.float32, shape=[1, 300, 400, 3])
with slim.arg_scope(resnet_v1.resnet_arg_scope()):
    net, end_points = resnet_v1.resnet_v1_50(inputs, is_training=False)
saver = tf.train.Saver()    

with tf.Session() as sess:
    saver.restore(sess, './resnet_v1_50.ckpt')
    # Resnet-50 layer names: https://github.com/KaimingHe/deep-residual-networks/blob/master/prototxt/ResNet-50-deploy.prototxt
    representation_tensor = sess.graph.get_tensor_by_name('resnet_v1_50/pool5:0') 
    for obs in observation:
        img = obs[np.newaxis,:]
        features = sess.run(representation_tensor, {'Placeholder:0': img})
        resnet_feature.append(features.flatten()[:,np.newaxis])

INFO:tensorflow:Restoring parameters from ./resnet_v1_50.ckpt


In [11]:
print('num of id:', len(resnet_feature),'   dimension:', resnet_feature[0].shape)
print('resnet_feature built from Resnet50')

num of id: 136    dimension: (2048, 1)
resnet_feature built from Resnet50


In [12]:
import h5py
import os
if os.path.exists(scene+'.h5'):
    os.remove(scene+'.h5')   
# write h5
f = h5py.File(scene+'.h5', 'w')
f['observation'] = observation
f['resnet_feature'] = resnet_feature
f['location'] = location
f['rotation'] = rotation
f['graph'] = graph
f.create_dataset('shortest_path_distance', data=shortest_path_distance)
f.close()

In [14]:
# test
#h5 = h5py.File(scene+'.h5', 'r')
#list(h5.keys())
#print(h5['location'][0])
#print(h5['rotation'][3])
#imgplot = plt.imshow(h5['observation'][2])
#type(h5['location'])
#print(h5['shortest_path_distance'][0][1])